In [ ]:
library(dplyr)
library(data.table)
library(tidyr)
library(taxizedb)
library(ggplot2)

In [ ]:
get_lineages <- function(x){
    # Print the fetched data
    #tax_info
    tt <- lapply(x, function(x){
        tax_info <- classification(x, db="ncbi")  
        df <- tax_info[[1]]
        if (is.data.frame(df)) {
                df$query <- x
                return(df)
            } else {
                return(NA)
            }

    })
    tt <- tt[!is.na(tt)]
    standard_ranks <- c("kingdom", "phylum", "class", "order", "family", "genus", "species")
    dfo <- bind_rows(tt) %>%
        filter(rank %in% standard_ranks)
    #     select(-id) %>%
    #     pivot_wider(names_from = rank, values_from = name)
    return(dfo)
}


In [ ]:
fin = paste(snakemake@input$search,paste(snakemake@wildcards$stem,".tsv",sep=""),sep="/")
df_match = fread(cmd = paste("grep -v '#' ",fin)) %>%
    select(taxid=V6) %>%
    group_by(taxid) %>%
    summarise(N=n()) %>%
    ungroup() %>%
    arrange(-N)

limit = df_match$N
#ggplot(df_match) + geom_histogram(aes(x=N))
l = quantile(x = limit,probs = 0.95)[[1]]
df_match <- df_match %>%
    filter(N > l)
lngs <- get_lineages(df_match$taxid)
lngs

In [ ]:
lngs2 <- lngs %>%
    mutate(name2 = paste(paste(name,id),sep="_")) %>%
    select(-name, -id) %>%
    pivot_wider(names_from = rank, values_from = name2) %>%
    select(-species, -query) %>%
    group_by(phylum,class, order, family, genus) %>%
    summarise(N=n()) %>%
    arrange(-N) %>%
    filter(N > 1)

In [ ]:
lngs2$bin <- sub(pattern = "nod_", replacement = "", x =  snakemake@wildcards$stem ) 
#df <- fread(file = snakemake@output$ass)
lngs2 <- lngs2 %>%
    arrange(-N)
fwrite(file = snakemake@output[[1]], x = lngs2)

In [ ]:
# lns = readLines(snakemake@input[[1]])
# ids = grepl(pattern = "Auto-inferred", x = lns, fixed = T)
# # Find indices where ids is TRUE
# true_indices <- which(ids)

# # Increment indices to get the next entries
# next_indices <- true_indices + 1

# # Ensure the indices do not exceed the number of rows in df
# next_indices <- next_indices[next_indices <= length(lns)]

# # Get the corresponding lines using the next indices
# df <- data.frame(matches = lns[next_indices]) %>%
#     separate(col = matches,sep = " \\(", into = c("A","B")) %>%
#     mutate(B = sub(pattern = ")", x = B, replacement="")) %>%
#     rename(taxid=A, name = B) %>%
#     mutate(type=c("self","close"))
# df